# Predicting Satisfiability of SAT-3 Problems
## Solving the Classification Task using a ***GNN***

A *satisfiability problem* (SAT problem) is a family of problems, where given a Boolean expression written using only the logical connectives AND, OR, NOT, variables and parentheses, we examine if there is some truth-assignment to the variables that will make the entire expression true. A **SAT-3** problem is a special case of SAT problem, where Boolean expression should have very strict form : the conjunctive normal form (CNF). A CNF is a conjunction of one or more clauses, where a clause is a disjunction of literals; otherwise put, it is an AND of ORs. In the SAT-3 problem, every disjunction consists of 3 literals. An example of a SAT-3 problem instance is the following:
$$(x_1 \vee x_2 \vee \neg x_4) \wedge (\neg x_3 \vee x_4 \vee x_2)$$

The SAT-3 problem is NP-complete and many solvers have been developed in order to prove the (un)satisfiability of the problems and point a truth assignment of the variables. However, not many large instances can be solved by traditional solvers and, thus, some researchers have come up with the idea to use Machine Learning methods in order to solve those problems, given that with the models we cannot have a guaranteed accuracy, but if the model generalizes well, then we would be able to solve some difficult problem-instances that the state-of-the-art solvers cannot.<br>

In this assignment we will try predicting the satisfiability of the instances using a *GNN* and a *Long Short-Term Memory network (LSTM)*. In this notebook will be explained the process of using a GNN and specifically a Graph Transformer.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
__counter__ = random.randint(0,2e9)

from IPython.display import HTML, display

In [3]:
import os, shutil
import json

from tuning import tune_parameters
from data_loader import dataset_processing
from train import training, testing

Device is: cuda:0


A helper function to ensure that the dataset is made from scratch when NN is tested. 

In [4]:
def delete_folder_contents(folders):
    for folder in folders:
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)

### Test set sampled from the same distribution as the training and validation sets 

In order to train the model, a *training set* is used alongside with a *validation set*. The latter is used to measure the performance of the model. The training and validation set were extracted from the 80% (60%-20%) of the data and the rest 20% was kept to evaluate the model after its training (*test set*).  

Erase the previous dataset created by PyTorch

In [5]:
delete_folder_contents(["./raw", "./processed"])

**Create the dataset**: The dataset is created from the raw data in such a format that it can be loader by the *DataLoader* module of torch.geometric. <br>

The dataset must be provided to Pytorch in a graph format, so in this preprocessing are constructed the *vertices, the edges, the labels* etc.<br>

The modelling of the classification problem to a graph problem will be explained by providing an example. The SAT-3 problem:
$$(x_1 \vee x_2 \vee \neg x_3) \wedge (\neg x_3 \vee \neg x_1 \vee x_2)$$

is translated to the following graph 

<img src="./plots/graph.jpg" height=250 width=300>

<b>Note:</b> The edges with different color connect different vertices and are considered as such in the *torch.geometric* format.

In [32]:
pos_weight = dataset_processing(separate_test=False)

Start the data processing...

Satisfiable CNFs   : 3701
Unsatisfiable CNFs : 2700

Ratio of SAT   : 0.5782
Ratio of UNSAT : 0.4218

Training set size: 5120
Test set size: 1280

Processing completed.


**Tune parameters**: Tune the parameters of the model as well as parameters regarding the training process e.g. *batch-size*, *weight decay* etc. More specifically, the parameters that are tuned are the following.
<ol><li>Model parameters:</li>
    <ul> 
        <li>Number of layers</li>
        <li>Dropout rate</li>
        <li>Neurons' density in its last Linear Layers</li>
        <li>Embedding size for the Graph Transformer Operator (TransformerConv)</li>
        <li>Attention heads for the Graph Transformer Operator (TransformerConv)</li><br>
    </ul>
    <li>Training parameters:</li>
    <ul> 
        <li>Batch size used</li>
        <li>Learning rate</li>
        <li>Weight decay</li>
    </ul>
</ol>

The parameters of the final model are the ones that result in the minimum ***validation error***. <br>
Also, ***early stopping*** is used in order to avoid *overfitting*.

In [7]:
# Tune parameters
best_parameters = tune_parameters(pos_weight=pos_weight, model_name='./final_model_same_sets.pth')

# Show best parameters
print(f'Best hyperparameters were: {best_parameters}')
# Store best parameters
with open('./best_parameters_same_sets.txt', 'w') as f:
    f.write(json.dumps(best_parameters))

f.close()


Test number 1 | Start testing new parameter-combination...

Dataset loading...


Processing...
100%|█████████████████████████████████████████████████████████████████████████████| 5120/5120 [00:10<00:00, 484.83it/s]
Done!


Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.6187
Validation Loss : 1.2107

EPOCH | 1
Training Loss   : 0.4662
Validation Loss : 0.6492

EPOCH | 2
Training Loss   : 0.4028
Validation Loss : 0.3204

EPOCH | 3
Training Loss   : 0.3620
Validation Loss : 0.6466

EPOCH | 4
Training Loss   : 0.3380
Validation Loss : 0.2776

EPOCH | 5
Training Loss   : 0.3717
Validation Loss : 0.4033

EPOCH | 6
Training Loss   : 0.3077
Validation Loss : 0.4027

EPOCH | 7
Training Loss   : 0.3186
Validation Loss : 0.8683

EPOCH | 8
Training Loss   : 0.3061
Validation Loss : 0.2687

EPOCH | 9
Training Loss   : 0.2917
Validation Loss : 0.6083

EPOCH | 10
Training Loss   : 0.2860
Validation Loss : 0.2515

EPOCH | 11
Training Loss   : 0.2814
Validation Loss : 0.2744

EPOCH | 12
Training Loss   : 0.2652
Validation Loss : 0.2564

EPOCH | 13
Training Loss   : 0.2668
Validation Loss : 0.2398

EPOCH | 14
Training Loss   : 0.2605
Validation Loss : 0.3035

EPOCH | 15


Validation Loss : 2.1244

EPOCH | 33
Training Loss   : 0.3193
Validation Loss : 1.4807

EPOCH | 34
Training Loss   : 0.3121
Validation Loss : 1.3726

EPOCH | 35
Early stopping activated, with training and validation loss difference: 0.0026

Test number 4 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.6775
Validation Loss : 2.0918

EPOCH | 1
Training Loss   : 0.5016
Validation Loss : 2.3265

EPOCH | 2
Training Loss   : 0.4539
Validation Loss : 4.4875

EPOCH | 3
Training Loss   : 0.4423
Validation Loss : 2.6863

EPOCH | 4
Training Loss   : 0.4320
Validation Loss : 0.4592

EPOCH | 5
Training Loss   : 0.3742
Validation Loss : 0.5058

EPOCH | 6
Training Loss   : 0.3393
Validation Loss : 4.0338

EPOCH | 7
Training Loss   : 0.3224
Validation Loss : 0.4126

EPOCH | 8
Training Loss   : 0.3203
Validation Loss : 1.1450

EPOCH | 9
Training Loss   : 0.2941
Validation Loss : 1.6419

E

Validation Loss : 0.5783

EPOCH | 3
Training Loss   : 0.4451
Validation Loss : 5.4630

EPOCH | 4
Training Loss   : 0.4610
Validation Loss : 0.9822

EPOCH | 5
Training Loss   : 0.4162
Validation Loss : 7.5645

EPOCH | 6
Training Loss   : 0.3857
Validation Loss : 0.3630

EPOCH | 7
Training Loss   : 0.3733
Validation Loss : 5.0792

EPOCH | 8
Training Loss   : 0.3956
Validation Loss : 6.2904

EPOCH | 9
Training Loss   : 0.3985
Validation Loss : 6.3524

EPOCH | 10
Training Loss   : 0.4206
Validation Loss : 8.9980

EPOCH | 11
Training Loss   : 0.4221
Validation Loss : 9.3508

EPOCH | 12
Training Loss   : 0.3762
Validation Loss : 2.8337

EPOCH | 13
Training Loss   : 0.3413
Validation Loss : 7.2489

EPOCH | 14
Training Loss   : 0.2764
Validation Loss : 0.4734

EPOCH | 15
Training Loss   : 0.2059
Validation Loss : 0.3986

EPOCH | 16
Training Loss   : 0.1774
Validation Loss : 0.1452

EPOCH | 17
Training Loss   : 0.1656
Validation Loss : 1.0359

EPOCH | 18
Training Loss   : 0.1436
Validation Loss

...

Training Loss   : 0.4760
Validation Loss : 3.7648

EPOCH | 4
Training Loss   : 0.4718
Validation Loss : 3.4493

EPOCH | 5
Training Loss   : 0.4561
Validation Loss : 0.8382

EPOCH | 6
Training Loss   : 0.4814
Validation Loss : 0.9541

EPOCH | 7
Training Loss   : 0.4382
Validation Loss : 0.9251

EPOCH | 8
Training Loss   : 0.4648
Validation Loss : 0.4812

EPOCH | 9
Training Loss   : 0.4862
Validation Loss : 5.3671

EPOCH | 10
Training Loss   : 0.4656
Validation Loss : 5.5664

EPOCH | 11
Training Loss   : 0.4368
Validation Loss : 7.1532

EPOCH | 12
Training Loss   : 0.4455
Validation Loss : 4.8443

EPOCH | 13
Training Loss   : 0.4328
Validation Loss : 1.8483

EPOCH | 14
Training Loss   : 0.4373
Validation Loss : 6.6998

EPOCH | 15
Training Loss   : 0.4235
Validation Loss : 5.9723

EPOCH | 16
Training Loss   : 0.4020
Validation Loss : 6.3451

EPOCH | 17
Training Loss   : 0.4131
Validation Loss : 6.9304

EPOCH | 18
Early stopping activated, with training and validation loss difference: 0.00

Training Loss   : 0.1490
Validation Loss : 0.1739

EPOCH | 32
Early stopping activated, with training and validation loss difference: 0.0007

Test number 40 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.6059
Validation Loss : 0.8289

EPOCH | 1
Training Loss   : 0.5147
Validation Loss : 2.5250

EPOCH | 2
Training Loss   : 0.5394
Validation Loss : 0.5862

EPOCH | 3
Training Loss   : 0.5851
Validation Loss : 0.5862

EPOCH | 4
Training Loss   : 0.5849
Validation Loss : 0.5862

EPOCH | 5
Training Loss   : 0.5849
Validation Loss : 0.5862

EPOCH | 6
Training Loss   : 0.5849
Validation Loss : 0.5862

EPOCH | 7
Training Loss   : 0.5849
Validation Loss : 0.5862

EPOCH | 8
Training Loss   : 0.5849
Validation Loss : 0.5862

EPOCH | 9
Training Loss   : 0.5849
Validation Loss : 0.5862

EPOCH | 10
Training Loss   : 0.5849
Validation Loss : 0.5862

EPOCH | 11
Training Loss   : 0.5848
V

Training Loss   : 0.5677
Validation Loss : 0.5660

EPOCH | 8
Training Loss   : 0.5752
Validation Loss : 0.7968

EPOCH | 9
Training Loss   : 0.5860
Validation Loss : 0.5987

EPOCH | 10
Training Loss   : 0.5791
Validation Loss : 0.9257

EPOCH | 11
Training Loss   : 0.5476
Validation Loss : 0.7927

EPOCH | 12
Training Loss   : 0.5454
Validation Loss : 0.5869

EPOCH | 13
Training Loss   : 0.5851
Validation Loss : 0.5868

EPOCH | 14
Training Loss   : 0.5851
Validation Loss : 0.5867

EPOCH | 15
Training Loss   : 0.5851
Validation Loss : 0.5866

EPOCH | 16
Training Loss   : 0.5851
Validation Loss : 0.5865

EPOCH | 17
Training Loss   : 0.5851
Validation Loss : 0.5864

EPOCH | 18
Training Loss   : 0.5851
Validation Loss : 0.5864

EPOCH | 19
Training Loss   : 0.5851
Validation Loss : 0.5863

EPOCH | 20
Training Loss   : 0.5851
Validation Loss : 0.5863

EPOCH | 21
Training Loss   : 0.5850
Validation Loss : 0.5862

EPOCH | 22
Training Loss   : 0.5850
Validation Loss : 0.5862

EPOCH | 23
Training L

Training Loss   : 0.2063
Validation Loss : 0.4598

EPOCH | 19
Early stopping activated, with training and validation loss difference: 0.0076

Test number 49 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.6228
Validation Loss : 1.4771

EPOCH | 1
Training Loss   : 0.4505
Validation Loss : 2.2878

EPOCH | 2
Training Loss   : 0.3837
Validation Loss : 0.4104

EPOCH | 3
Training Loss   : 0.3535
Validation Loss : 0.6020

EPOCH | 4
Training Loss   : 0.3487
Validation Loss : 6.4646

EPOCH | 5
Training Loss   : 0.3231
Validation Loss : 0.6198

EPOCH | 6
Training Loss   : 0.3048
Validation Loss : 1.5513

EPOCH | 7
Training Loss   : 0.3366
Validation Loss : 1.0615

EPOCH | 8
Training Loss   : 0.2736
Validation Loss : 0.2978

EPOCH | 9
Training Loss   : 0.2535
Validation Loss : 0.2422

EPOCH | 10
Training Loss   : 0.2453
Validation Loss : 0.2149

EPOCH | 11
Training Loss   : 0.2231
V

Training Loss   : 0.5900
Validation Loss : 0.5874

EPOCH | 3
Training Loss   : 0.5853
Validation Loss : 0.5870

EPOCH | 4
Training Loss   : 0.5853
Validation Loss : 0.5869

EPOCH | 5
Training Loss   : 0.5853
Validation Loss : 0.5868

EPOCH | 6
Training Loss   : 0.5853
Validation Loss : 0.5867

EPOCH | 7
Training Loss   : 0.5853
Validation Loss : 0.5866

EPOCH | 8
Training Loss   : 0.5853
Validation Loss : 0.5865

EPOCH | 9
Training Loss   : 0.5853
Validation Loss : 0.5864

EPOCH | 10
Training Loss   : 0.5854
Validation Loss : 0.5863

EPOCH | 11
Training Loss   : 0.5853
Validation Loss : 0.5862

EPOCH | 12
Training Loss   : 0.5853
Validation Loss : 0.5862

EPOCH | 13
Training Loss   : 0.5853
Validation Loss : 0.5862

EPOCH | 14
Training Loss   : 0.5853
Validation Loss : 0.5862

EPOCH | 15
Training Loss   : 0.5852
Validation Loss : 0.5861

EPOCH | 16
Training Loss   : 0.5852
Validation Loss : 0.5861

EPOCH | 17
Training Loss   : 0.5852
Validation Loss : 0.5861

EPOCH | 18
Training Loss  

**Train with the best parameters**: The NN is trained using the selected parameters.

In [33]:
# Access the best parameters in order to train final model
with open('./best_parameters_same_sets.txt') as f:
    data = f.read()

best_parameters_loaded = json.loads(data)

print('\nNow training with the best parameters\n')
training(params=best_parameters_loaded, model_name='./final_model_same_sets.pth', make_err_logs=True)


Now training with the best parameters

Dataset loading...


Processing...
100%|█████████████████████████████████████████████████████████████████████████████| 5120/5120 [00:08<00:00, 617.42it/s]
Done!


Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.5687
Validation Loss : 1.8299

EPOCH | 1
Training Loss   : 0.3965
Validation Loss : 1.4843

EPOCH | 2
Training Loss   : 0.3532
Validation Loss : 0.8844

EPOCH | 3
Training Loss   : 0.3327
Validation Loss : 1.3748

EPOCH | 4
Training Loss   : 0.2848
Validation Loss : 0.5043

EPOCH | 5
Training Loss   : 0.2416
Validation Loss : 0.4693

EPOCH | 6
Training Loss   : 0.2226
Validation Loss : 0.3526

EPOCH | 7
Training Loss   : 0.2109
Validation Loss : 1.4364

EPOCH | 8
Training Loss   : 0.2073
Validation Loss : 1.8730

EPOCH | 9
Training Loss   : 0.2053
Validation Loss : 4.2894

EPOCH | 10
Training Loss   : 0.1843
Validation Loss : 2.9224

EPOCH | 11
Training Loss   : 0.1672
Validation Loss : 0.4713

EPOCH | 12
Training Loss   : 0.1582
Validation Loss : 0.9919

EPOCH | 13
Training Loss   : 0.1665
Validation Loss : 0.4200

EPOCH | 14
Training Loss   : 0.1531
Validation Loss : 0.6056

EPOCH | 15


0.13717702124267817

The following Figure shows the ***training and validation set errors***, the epoch where the ***early stopping*** was activated, as well as the epoch of the final ***selected model***.

In [34]:
display(HTML('<img src="plots/train_valid_error.png?%d" height=400 width=400>' % __counter__))

**Test the model**: Predict the satisfiability of the clauses in the *Test Set* and get the corresponding metrics.

In [35]:
print('\nResults on the test set:\n')
testing(params=best_parameters_loaded, plot_var="same", model_name='./final_model_same_sets.pth',)


Results on the test set:

Dataset loading...


Processing...
100%|█████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:01<00:00, 654.35it/s]
Done!


Dataset loading completed

Model loading...
Model loading completed


Test set metrics:

 Confusion matrix: 
 [[535 102]
 [ 13 630]]
F1 Score  : 0.9164
Accuracy  : 0.9102
Precision : 0.9798
Recall    : 0.8607
ROC AUC   : 0.9185
Test Loss : 0.13191640627519519


The following **Figures** show:
<ol>
<li>The <b>confusion matrix</b> for the test set-prediction</li>
<li>The <b>ROC-AUC curve</b> for the test set-prediction</li>
<li>The <b>precision recall curve</b> for the test set-prediction</li>
</ol>

In [36]:
print("\n1.")
display(HTML('<img src="plots/cm_same.png?%d" height=500 width=500>' % __counter__))
print("2.")
display(HTML('<img src="plots/roc_auc_same.png?%d" height=450 width=450>' % __counter__))
print("3.")
display(HTML('<img src="plots/pr_same.png?%d" height=450 width=450>' % __counter__))


1.


2.


3.


### Test set sampled from a different distribution from the training and validation sets 

Here, again, in order to train the model, a training set is used alongside with a validation set. However, now, they are both sampled randomly from the smaller 3-SAT problems i.e. have less than 250 variables. The test set consists of 200 problem instances that have 250 variables and 1065 clauses each. Since in this case we would like to predict the satisfiability of the problems that the well known algorithms cannot predict, ***we want to measure the performance of the model for bigger instances that it has not seen during its training***.

The training-tuning process is the same as showcased above.

In [6]:
# Erase the previous dataset created by PyTorch
delete_folder_contents(["./raw", "./processed"])

In [7]:
# Create the dataset
pos_weight = dataset_processing(separate_test=True)

Start the data processing...

Satisfiable CNFs   : 3601
Unsatisfiable CNFs : 2600

Ratio of SAT   : 0.5807
Ratio of UNSAT : 0.4193

Training set size: 6200
Test set size: 200

Processing completed.


In [22]:
# Tune parameters
best_parameters = tune_parameters(pos_weight=pos_weight, model_name='./final_model_diff_sets.pth')

# Show best parameters
print(f'Best hyperparameters were: {best_parameters}')
# Store best parameters
with open('best_parameters_diff_test.txt', 'w') as f:
    f.write(json.dumps(best_parameters))

f.close()


Test number 1 | Start testing new parameter-combination...

Dataset loading...


Processing...
100%|█████████████████████████████████████████████████████████████████████████████| 6200/6200 [00:10<00:00, 566.62it/s]
Done!


Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.5904
Validation Loss : 0.5819

EPOCH | 1
Training Loss   : 0.5794
Validation Loss : 0.5843

EPOCH | 2
Training Loss   : 0.5674
Validation Loss : 0.5801

EPOCH | 3
Training Loss   : 0.5807
Validation Loss : 0.5849

EPOCH | 4
Training Loss   : 0.5778
Validation Loss : 0.5853

EPOCH | 5
Training Loss   : 0.5723
Validation Loss : 0.5832

EPOCH | 6
Training Loss   : 0.5677
Validation Loss : 0.5982

EPOCH | 7
Training Loss   : 0.5714
Validation Loss : 0.6279

EPOCH | 8
Training Loss   : 0.5636
Validation Loss : 0.9375

EPOCH | 9
Training Loss   : 0.5631
Validation Loss : 1.1023

EPOCH | 10
Training Loss   : 0.5581
Validation Loss : 1.6557

EPOCH | 11
Training Loss   : 0.5338
Validation Loss : 1.0792

EPOCH | 12
Training Loss   : 0.5049
Validation Loss : 1.9030

EPOCH | 13
Training Loss   : 0.4946
Validation Loss : 1.5289

EPOCH | 14
Training Loss   : 0.4892
Validation Loss : 0.9247

EPOCH | 15


Validation Loss : 0.6092

EPOCH | 6
Training Loss   : 0.2868
Validation Loss : 0.3836

EPOCH | 7
Training Loss   : 0.2402
Validation Loss : 1.8599

EPOCH | 8
Training Loss   : 0.2252
Validation Loss : 1.3519

EPOCH | 9
Training Loss   : 0.1798
Validation Loss : 0.4649

EPOCH | 10
Training Loss   : 0.1719
Validation Loss : 0.3429

EPOCH | 11
Training Loss   : 0.1614
Validation Loss : 0.6132

EPOCH | 12
Training Loss   : 0.1666
Validation Loss : 0.3612

EPOCH | 13
Training Loss   : 0.1600
Validation Loss : 0.2265

EPOCH | 14
Training Loss   : 0.1380
Validation Loss : 0.2640

EPOCH | 15
Training Loss   : 0.1391
Validation Loss : 0.2629

EPOCH | 16
Training Loss   : 0.1308
Validation Loss : 0.3112

EPOCH | 17
Early stopping activated, with training and validation loss difference: 0.0456

Test number 7 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.5740
Validation Loss : 0.79

Validation Loss : 1.9233

EPOCH | 8
Training Loss   : 0.1685
Validation Loss : 2.0081

EPOCH | 9
Training Loss   : 0.3039
Validation Loss : 1.1955

EPOCH | 10
Training Loss   : 0.2170
Validation Loss : 2.9963

EPOCH | 11
Training Loss   : 0.1798
Validation Loss : 3.1299

EPOCH | 12
Training Loss   : 0.1520
Validation Loss : 2.7204

EPOCH | 13
Training Loss   : 0.1367
Validation Loss : 3.3644

EPOCH | 14
Training Loss   : 0.1307
Validation Loss : 3.2037

EPOCH | 15
Training Loss   : 0.1251
Validation Loss : 3.3391

EPOCH | 16
Early stopping activated, with training and validation loss difference: 0.0368

Test number 10 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.7114
Validation Loss : 0.6322

EPOCH | 1
Training Loss   : 0.5092
Validation Loss : 6.6517

EPOCH | 2
Training Loss   : 0.4693
Validation Loss : 5.7779

EPOCH | 3
Training Loss   : 0.4576
Validation Loss : 0.42

...

Training Loss   : 0.4821
Validation Loss : 0.4913

EPOCH | 24
Training Loss   : 0.4923
Validation Loss : 0.4896

EPOCH | 25
Training Loss   : 0.4920
Validation Loss : 0.4894

EPOCH | 26
Training Loss   : 0.4919
Validation Loss : 0.4893

EPOCH | 27
Training Loss   : 0.4918
Validation Loss : 0.4892

EPOCH | 28
Training Loss   : 0.4918
Validation Loss : 0.4892

EPOCH | 29
Training Loss   : 0.4917
Validation Loss : 0.4891

EPOCH | 30
Training Loss   : 0.4917
Validation Loss : 0.4891

EPOCH | 31
Training Loss   : 0.4917
Validation Loss : 0.4891

EPOCH | 32
Training Loss   : 0.4917
Validation Loss : 0.4891

EPOCH | 33
Training Loss   : 0.4917
Validation Loss : 0.4891

EPOCH | 34
Training Loss   : 0.4917
Validation Loss : 0.4891

EPOCH | 35
Training Loss   : 0.4917
Validation Loss : 0.4891

EPOCH | 36
Training Loss   : 0.4917
Validation Loss : 0.4891

EPOCH | 37
Early stopping activated, with training and validation loss difference: 0.0001

Test number 34 | Start testing new parameter-combina

Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.5635
Validation Loss : 0.5158

EPOCH | 1
Training Loss   : 0.5092
Validation Loss : 2.7623

EPOCH | 2
Training Loss   : 0.4920
Validation Loss : 0.5029

EPOCH | 3
Training Loss   : 0.3977
Validation Loss : 0.9620

EPOCH | 4
Training Loss   : 0.1578
Validation Loss : 2.7651

EPOCH | 5
Training Loss   : 0.1133
Validation Loss : 0.4609

EPOCH | 6
Training Loss   : 0.0869
Validation Loss : 0.2116

EPOCH | 7
Training Loss   : 0.0824
Validation Loss : 0.0698

EPOCH | 8
Training Loss   : 0.0747
Validation Loss : 0.0846

EPOCH | 9
Training Loss   : 0.0726
Validation Loss : 0.0841

EPOCH | 10
Training Loss   : 0.0724
Validation Loss : 0.0855

EPOCH | 11
Training Loss   : 0.0713
Validation Loss : 0.0656

EPOCH | 12
Training Loss   : 0.0651
Validation Loss : 0.0656

EPOCH | 13
Training Loss   : 0.0589
Validation Loss : 0.0563

EPOCH | 14
Training Loss   : 0.0552
Validation Loss : 0.0725

EPOCH | 15


Training Loss   : 0.5276
Validation Loss : 0.6291

EPOCH | 19
Training Loss   : 0.5266
Validation Loss : 0.5794

EPOCH | 20
Training Loss   : 0.5256
Validation Loss : 0.5958

EPOCH | 21
Training Loss   : 0.5228
Validation Loss : 0.5780

EPOCH | 22
Early stopping activated, with training and validation loss difference: 0.0002

Test number 43 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 3.0426
Validation Loss : 0.5817

EPOCH | 1
Training Loss   : 0.5823
Validation Loss : 0.5815

EPOCH | 2
Training Loss   : 0.5823
Validation Loss : 0.5815

EPOCH | 3
Training Loss   : 0.5822
Validation Loss : 0.5815

EPOCH | 4
Training Loss   : 0.5822
Validation Loss : 0.5816

EPOCH | 5
Training Loss   : 0.5821
Validation Loss : 0.5817

EPOCH | 6
Training Loss   : 0.5821
Validation Loss : 0.5817

EPOCH | 7
Training Loss   : 0.5821
Validation Loss : 0.5817

EPOCH | 8
Training Loss   : 0.5820


Training Loss   : 0.5595
Validation Loss : 0.6388

EPOCH | 1
Training Loss   : 0.5186
Validation Loss : 0.5935

EPOCH | 2
Training Loss   : 0.4974
Validation Loss : 0.8928

EPOCH | 3
Training Loss   : 0.4379
Validation Loss : 0.8955

EPOCH | 4
Training Loss   : 0.2102
Validation Loss : 0.8272

EPOCH | 5
Training Loss   : 0.1394
Validation Loss : 0.4626

EPOCH | 6
Training Loss   : 0.1201
Validation Loss : 0.1961

EPOCH | 7
Training Loss   : 0.1144
Validation Loss : 0.1882

EPOCH | 8
Training Loss   : 0.0995
Validation Loss : 0.0906

EPOCH | 9
Training Loss   : 0.0820
Validation Loss : 0.1076

EPOCH | 10
Training Loss   : 0.1071
Validation Loss : 0.0806

EPOCH | 11
Training Loss   : 0.0718
Validation Loss : 0.0599

EPOCH | 12
Training Loss   : 0.0515
Validation Loss : 0.0342

EPOCH | 13
Training Loss   : 0.0329
Validation Loss : 0.1108

EPOCH | 14
Training Loss   : 0.0251
Validation Loss : 0.0691

EPOCH | 15
Training Loss   : 0.0217
Validation Loss : 0.0326

EPOCH | 16
Training Loss   :

Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 7.6336
Validation Loss : 0.5272

EPOCH | 1
Training Loss   : 0.5817
Validation Loss : 0.5201

EPOCH | 2
Training Loss   : 0.5819
Validation Loss : 0.5181

EPOCH | 3
Training Loss   : 0.5819
Validation Loss : 0.5250

EPOCH | 4
Training Loss   : 0.5819
Validation Loss : 0.5612

EPOCH | 5
Training Loss   : 0.5819
Validation Loss : 0.5953

EPOCH | 6
Training Loss   : 0.5819
Validation Loss : 0.6069

EPOCH | 7
Training Loss   : 0.5819
Validation Loss : 0.6042

EPOCH | 8
Training Loss   : 0.5818
Validation Loss : 0.6001

EPOCH | 9
Training Loss   : 0.5818
Validation Loss : 0.5960

EPOCH | 10
Training Loss   : 0.5818
Validation Loss : 0.5916

EPOCH | 11
Training Loss   : 0.5818
Validation Loss : 0.5862

EPOCH | 12
Training Loss   : 0.5818
Validation Loss : 0.5796

EPOCH | 13
Training Loss   : 0.5817
Validation Loss : 0.5760

EPOCH | 14
Training Loss   : 0.5817
Validation Loss : 0.5760

EPOCH | 15


In [8]:
# Train with the best parameters

# Access the best parameters in order to train final model
with open('best_parameters_diff_test.txt') as f:
    data = f.read()

best_parameters_loaded = json.loads(data)

print('\nNow training with the best parameters\n')
training(params=best_parameters_loaded, model_name='./final_model_diff_sets.pth')


Now training with the best parameters

Dataset loading...


Processing...
100%|█████████████████████████████████████████████████████████████████████████████| 6200/6200 [00:08<00:00, 694.03it/s]
Done!


Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.5801
Validation Loss : 0.5717

EPOCH | 1
Training Loss   : 0.5563
Validation Loss : 0.5800

EPOCH | 2
Training Loss   : 0.5379
Validation Loss : 0.6364

EPOCH | 3
Training Loss   : 0.5162
Validation Loss : 0.5630

EPOCH | 4
Training Loss   : 0.4984
Validation Loss : 0.6375

EPOCH | 5
Training Loss   : 0.4801
Validation Loss : 0.4205

EPOCH | 6
Training Loss   : 0.4195
Validation Loss : 0.5330

EPOCH | 7
Training Loss   : 0.2866
Validation Loss : 0.8748

EPOCH | 8
Training Loss   : 0.1395
Validation Loss : 0.2764

EPOCH | 9
Training Loss   : 0.0854
Validation Loss : 0.4214

EPOCH | 10
Training Loss   : 0.0645
Validation Loss : 0.0677

EPOCH | 11
Training Loss   : 0.0542
Validation Loss : 0.1114

EPOCH | 12
Training Loss   : 0.0558
Validation Loss : 0.0431

EPOCH | 13
Training Loss   : 0.0421
Validation Loss : 0.1976

EPOCH | 14
Training Loss   : 0.0361
Validation Loss : 0.1572

EPOCH | 15


0.0677272844582032

In [9]:
# Test the model
print('\nResults on the test set:\n')
testing(params=best_parameters_loaded, plot_var="diff", model_name='./final_model_diff_sets.pth')


Results on the test set:

Dataset loading...


Processing...
100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 357.67it/s]
Done!


Dataset loading completed

Model loading...
Model loading completed


Test set metrics:

 Confusion matrix: 
 [[69 50]
 [31 50]]
F1 Score  : 0.5525
Accuracy  : 0.5950
Precision : 0.6173
Recall    : 0.5000
ROC AUC   : 0.5950
Test Loss : 0.5732883640698024


The following **Figures** show:
<ol>
<li>The <b>confusion matrix</b> for the test set-prediction</li>
<li>The <b>ROC-AUC curve</b> for the test set-prediction</li>
<li>The <b>precision recall curve</b> for the test set-prediction</li>
</ol>

In [10]:
__counter__ = random.randint(0,2e9)
print("\n1.")
display(HTML('<img src="plots/cm_diff.png?%d" height=500 width=500>' % __counter__))
print("2.")
display(HTML('<img src="plots/roc_auc_diff.png?%d" height=450 width=450>' % __counter__))
print("3.")
display(HTML('<img src="plots/pr_diff.png?%d" height=450 width=450>' % __counter__))


1.


2.


3.
